Continuing work done in nyt_eda.ipynb
- extract byline information
- do sanity check to ensure that Author matches person or organization in byline dictionary in most/all rows

In [13]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import re


from ast import literal_eval

In [14]:
# load data
#front_pages=pd.read_csv('front_pages.zip',compression='zip', sep=',') 
rest = pd.read_csv('rest.zip', compression='zip', sep=',')
print(rest.shape)

with gzip.open('frontpages.zip', 'rb') as f:
    front_pages = pd.read_csv(f)
print(front_pages.shape)

C:\Users\scj41343\AppData\Local\Temp\ipykernel_6660\1158527711.py:3: DtypeWarning: Columns (5,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  rest = pd.read_csv('rest.zip', compression='zip', sep=',')


(392847, 31)
(21917, 27)


In [15]:
#remove underscore from column names as python was not liking them in byline_extraction function, not sure why
rest.columns = rest.columns.str.replace("_","")
front_pages.columns = front_pages.columns.str.replace("_","")
#determine N/A counts for rest df
rest.isna().sum()

abstract            1021
weburl                 1
snippet             6522
leadparagraph       4353
printsection      209304
printpage         209294
source                 1
multimedia             1
headline               1
keywords               1
pubdate                1
documenttype           1
newsdesk           12665
sectionname          328
subsectionname    239421
byline                 1
typeofmaterial      7590
id                     1
wordcount              1
uri                    1
year                   1
numsubjects            1
numpersons             1
numglocs               1
numcreatives           1
numorgs                1
glocs                  1
persons                1
subjects               2
orgs                   2
cworks                 2
dtype: int64

In [16]:
#determine N/A counts for front pages df
front_pages.isna().sum()

abstract             0
weburl               0
snippet              2
leadparagraph       66
printsection         0
printpage            0
source               0
multimedia           0
headline             0
keywords             0
pubdate              0
documenttype         0
newsdesk            13
sectionname          0
subsectionname    8844
byline               0
typeofmaterial       0
id                   0
wordcount            0
uri                  0
year                 0
numsubj              0
numpersons           0
numglocs             0
numcreatives         0
persons              0
subjects             0
dtype: int64

In [17]:
#drop one row where byline = 188 (int)
intbyline = rest['byline'] == '188'
rest = rest[~intbyline]

In [18]:
#drop rows with N/As
rest = rest[rest['byline'].notna()]
#reset index so df is clean still
rest = rest.reset_index(drop=True)
print(rest.shape)
print(rest.columns)

(392845, 31)
Index(['abstract', 'weburl', 'snippet', 'leadparagraph', 'printsection',
       'printpage', 'source', 'multimedia', 'headline', 'keywords', 'pubdate',
       'documenttype', 'newsdesk', 'sectionname', 'subsectionname', 'byline',
       'typeofmaterial', 'id', 'wordcount', 'uri', 'year', 'numsubjects',
       'numpersons', 'numglocs', 'numcreatives', 'numorgs', 'glocs', 'persons',
       'subjects', 'orgs', 'cworks'],
      dtype='object')


In [19]:
#drop rows with N/As (this was done before the N/A count for front pages was done, so it was learned that it was not necessary)
front_pages = front_pages[front_pages['byline'].notna()]
#reset index so df is clean still
front_pages = front_pages.reset_index(drop=True)
print(front_pages.shape)
print(front_pages.columns)


(21917, 27)
Index(['abstract', 'weburl', 'snippet', 'leadparagraph', 'printsection',
       'printpage', 'source', 'multimedia', 'headline', 'keywords', 'pubdate',
       'documenttype', 'newsdesk', 'sectionname', 'subsectionname', 'byline',
       'typeofmaterial', 'id', 'wordcount', 'uri', 'year', 'numsubj',
       'numpersons', 'numglocs', 'numcreatives', 'persons', 'subjects'],
      dtype='object')


In [20]:
def byline_extraction(df):
    #initialize empty columns to store extracted strings
    df['byline_extract'] = ""
    df['byline_confirm'] = ""
    df['byline_match'] = ""

    #for every row in the provided dataframe, turn string into dictionary
    #the author is extracted from the value of the original key, remove additional text to just get the name of the person or organization
    #only assign first author, so remove anything after 'and' or ','
    for row in range(0,df.shape[0]):
        dict = literal_eval(df['byline'].loc[row])
        author = dict['original']
        author = re.sub('By ','',author)
        author = re.sub(' By ','',author)
        author = re.sub('text by ','',author,flags=re.I)
        author = re.sub('interview by ','',author,flags=re.I)
        author = re.sub('compiled by ','',author,flags=re.I)
        author = re.sub('photographs by ','',author,flags=re.I)
        author = re.sub('text and photographs by ','',author,flags=re.I)
        if ' and' in author:
            author = author[:author.index(' and')]
        if ', ' in author:
            author = author[:author.index(', ')]
        #capitalize name correctly and remove and whitespace
        author = author.title()
        author = author.strip()
        #assign the byline_extract with the author
        df.loc[row,'byline_extract'] = author
        
        #if the person dictionary is empty, this indicates it is an organization as the author, use organization dict to confirm
        if dict['person'] == []:
            author_con = dict['organization']
            if not author_con:
                #if there is no organization dictionary, assign N/A to byline_confirm
                author_con = 'N/A'
            author_confirm = author_con
        else:
            author_con = dict['person']
            #get the byline information from the person dictionary in order to confirm that they match
            #if there is no person dictionary, assign N/A to byline_confirm
            if not author_con:
                author_con = 'N/A'
            else:
                #if there is a person dictionary, extract the first and last name, if there is a middle name, extract that as well
                #concatenate the first, possible middle, and last names (and qualifiers if needed ex: Jr)
                author_con = author_con[0]
                author_first = author_con['firstname']
                author_last = author_con['lastname']
                if author_con['middlename'] != None:
                    author_middle = author_con['middlename']
                    author_confirm = author_first + " " + author_middle + " " + author_last
                else:
                    author_confirm = author_first + " " + author_last
                if author_con['qualifier'] != None:
                    author_qual = author_con['qualifier']
                    author_confirm = author_confirm + " " + author_qual
            #remove additional works just to get the author
            author_confirm = re.sub('text ','',author_confirm,flags=re.I)
            author_confirm = re.sub('interview ','',author_confirm,flags=re.I)
            author_confirm = re.sub('photograph ','',author_confirm,flags=re.I)
            author_confirm = author_confirm.replace('M.D','')
        #capitalize name correctly and remove and whitespace
        author_confirm = author_confirm.title()
        author_confirm = author_confirm.strip()
        #assign name to byline_confirm
        df.loc[row,'byline_confirm'] = author_confirm
        
        #check to see if byline_extract and byline_confirm match
        if author == author_confirm:
            df.loc[row,'byline_match'] = "Yes"
        else:
            df.loc[row,'byline_match'] = "No"
    return df

In [21]:
rest = byline_extraction(rest)

In [22]:
front_pages = byline_extraction(front_pages)

In [23]:
print(front_pages['byline_match'].value_counts()['Yes'])
print(front_pages['byline_match'].value_counts()['No'])

21058
859


In [24]:
fpissues = front_pages[front_pages['byline_match'] == 'No']
fpissues = fpissues[['byline','byline_extract','byline_confirm']]
fpissues = fpissues.reset_index(drop=True)

In [25]:
fpissues.to_csv('fpissues.csv')

In [26]:
print(rest['byline_match'].value_counts()['Yes'])
print(rest['byline_match'].value_counts()['No'])

346336
46509


In [27]:
rissues = rest[rest['byline_match'] == 'No']
rissues = rissues[['byline','byline_extract','byline_confirm']]
rissues = rissues.reset_index(drop=True)

In [28]:
rissues.to_csv('rissues.csv')

In [29]:
rest.to_csv('rest_byline_cleaned.gz',compression = 'gzip')
front_pages.to_csv('front_pages_byline_cleaned.gz', compression = 'gzip')

In [30]:
rest.to_csv('rest_byline_cleaned.csv')
front_pages.to_csv('front_pages_byline_cleaned.csv',)